# Temporal analysis with PASTIS matrices

Note how this notebook requires the additional python library `exoscene` which is currently not included in the `environment.yml` file for the standard `pastis` conda environment.

In [ ]:
import os
import time

import numpy as np
from astropy.io import fits
import astropy.units as u
import astropy.constants as c
import hcipy

from pastis.config import CONFIG_PASTIS 
import pastis.util as util 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC

import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet

In [ ]:
print(' Loading basic parameters for this specific telescope \n')

### Parameters
design = 'small'

# System parameters
root_dir = CONFIG_PASTIS.get('local', 'local_data_path')
overall_dir = util.create_data_path(root_dir, telescope='luvoir_'+design)
# Moving parts parameters
analysis_name = 'LUVOIRA_APLC_' + design
max_LO = CONFIG_PASTIS.getint('dm_objects', 'number_of_low_order_modes')
max_MID = CONFIG_PASTIS.getint('dm_objects', 'number_of_mid_order_modes')
max_HI = CONFIG_PASTIS.getint('dm_objects', 'number_of_high_order_modes')
num_DM_act = CONFIG_PASTIS.getint('dm_objects', 'number_of_continuous_dm_actuators')
# General telescope parameters
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9   # m
# Image system parameters
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

In [ ]:
# Print some of the defined parameters
print('LUVOIR apodizer design: {}'.format(design))
print()
print('Wavelength: {} m'.format(wvln))
print('Telescope diameter: {} m'.format(diam))
print('Number of segments: {}'.format(nb_seg))
print()
print('Sampling: {} px per lambda/D'.format(sampling))

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, design, sampling)
npup = np.int(np.sqrt(luvoir.pupil_grid.x.shape[0]))
nimg = np.int(np.sqrt(luvoir.focal_det.x.shape[0]))

In [ ]:
# Load the matrices
filename_matrix = 'EFIELD_Re_matrix_num_LO_' + str(max_LO) +'.fits'
G_LO_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_Im_matrix_num_LO_' + str(max_LO) +'.fits'
G_LO_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix =  'EFIELD_Re_matrix_num_MID_' + str(max_MID) +'.fits'
G_MID_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix =  'EFIELD_Im_matrix_num_MID_' + str(max_MID) +'.fits'
G_MID_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix =  'EFIELD_Re_matrix_num_HI_' + str(max_HI) +'.fits'
G_HI_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix =  'EFIELD_Im_matrix_num_HI_' + str(max_HI) +'.fits'
G_HI_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_LOWFS_Re_matrix_num_LO_' + str(max_LO) +'.fits'
G_LOWFS_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_LOWFS_Im_matrix_num_LO_' + str(max_LO) +'.fits'
G_LOWFS_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_MID_' + str(max_MID) +'.fits'
G_OBWFS_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_MID_' + str(max_MID) +'.fits'
G_OBWFS_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

In [ ]:
luvoir.create_global_zernike_mirror(max_LO)
luvoir.create_segmented_mirror(max_MID)
luvoir.create_ripple_mirror(max_HI)
luvoir.create_continuous_deformable_mirror(num_DM_act)

n_LO = luvoir.zernike_mirror.num_actuators
n_MID = luvoir.sm.num_actuators 
n_HI = luvoir.ripple_mirror.num_actuators
n_DM = luvoir.dm.num_actuators

In [ ]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample')
N_pup_z = np.int(luvoir.pupil_grid.shape[0] / z_pup_downsample)
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

luvoir.zernike_mirror.actuators = LO_modes
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes
luvoir.dm.actuators = DM_modes

unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False)
norm = np.max(ref)
dh_intensity = (unaberrated_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print(f'contrast floor: {contrast_floor}')

nonaberrated_coro_psf, ref,inter_ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
# Make matrices
mat_LO = np.zeros([n_LO-1, n_LO-1])
for i in range(1, n_LO):
    for j in range(1, n_LO):
        tmpI = (G_LO_real[i]+1j*G_LO_imag[i] - Efield_ref)
        tmpJ = (G_LO_real[j]+1j*G_LO_imag[j] - Efield_ref)
        test = np.real(tmpI*np.conj(tmpJ))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_LO[i-1, j-1] = contrast

In [ ]:
mat_MID = np.zeros([n_MID, n_MID])
for i in range(0, n_MID):
    for j in range(0, n_MID):
        tmpI = G_MID_real[i]+1j*G_MID_imag[i] - Efield_ref
        tmpJ = G_MID_real[j]+1j*G_MID_imag[j] - Efield_ref
        test = np.real(tmpI*np.conj(tmpJ))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_MID[i, j] = contrast

In [ ]:
mat_HI = np.zeros([n_HI, n_HI])
for i in range(0, n_HI):
    for j in range(0, n_HI):
        tmpI = G_HI_real[i]+1j*G_HI_imag[i] - Efield_ref
        tmpJ = G_HI_real[j]+1j*G_HI_imag[j] - Efield_ref
        test = np.real(tmpI*np.conj(tmpJ))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_HI[i, j] = contrast

In [ ]:
# PASTIS eigenvalues for all scales

evalsLO, evecsLO = np.linalg.eig(mat_LO)
sorted_evalsLO = np.sort(evalsLO)
sorted_indicesLO = np.argsort(evalsLO)
sorted_evecsLO = evecsLO[:, sorted_indicesLO]

evalsMID, evecsMID = np.linalg.eig(mat_MID)
sorted_evalsMID = np.sort(evalsMID)
sorted_indicesMID = np.argsort(evalsMID)
sorted_evecsMID = evecsMID[:, sorted_indicesMID]

evalsHI, evecsHI = np.linalg.eig(mat_HI)
sorted_evalsHI = np.sort(evalsHI)
sorted_indicesHI = np.argsort(evalsHI)
sorted_evecsHI = evecsHI[:, sorted_indicesHI]

In [ ]:
# Calculate the segment based constraints
c_target_log = -11
c_target = 10**(c_target_log)
n_repeat = 20

In [ ]:
mu_mapLO = np.sqrt(((c_target) / (n_LO-1)) / (np.diag(mat_LO)))
mu_mapMID = np.sqrt(((c_target) / (n_MID)) / (np.diag(mat_MID)))
mu_mapHI = np.sqrt(((c_target) / (n_HI)) / (np.diag(mat_HI)))

In [ ]:
# Getting the flux together
sptype = 'A0V' # Put this on config
Vmag = 0.0 # Put this in loop
minlam = 500 * u.nanometer # Put this on config
maxlam = 600 * u.nanometer # Put this on config
star_flux = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=Vmag, minlam=minlam.value, maxlam=maxlam.value)
Nph = star_flux.value*15**2*np.sum(luvoir.apodizer**2) / npup**2
dark_current = 0.0002
CIC = 0.01
# dark_current = 0.0000
# CIC = 0.00

In [ ]:
# Reference fluxes for the WF sensors
# In particular we downsample everything
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

luvoir.zernike_mirror.actuators = LO_modes
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes
luvoir.dm.actuators = DM_modes

In [ ]:
nonaberrated_coro_psf, refshit,inter_ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
zernike_ref = luvoir.calc_low_order_wfs()
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref_LOWFS = (zernike_ref_sub_real + 1j*zernike_ref_sub_imag) * z_pup_downsample

In [ ]:
zernike_ref = luvoir.calc_out_of_band_wfs()
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref_OBWFS = (zernike_ref_sub_real + 1j*zernike_ref_sub_imag) * z_pup_downsample

In [ ]:
nyquist_sampling = 2.

# Actual grid for LUVOIR images
grid_test = hcipy.make_focal_grid(
            luvoir.sampling,
            luvoir.imlamD,
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )

# Actual grid for LUVOIR images that are nyquist sampled
grid_det_subsample = hcipy.make_focal_grid(
            nyquist_sampling,
            np.floor(luvoir.imlamD),
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )
n_nyquist = np.int(np.sqrt(grid_det_subsample.x.shape[0]))

In [ ]:
### Dark hole mask
dh_outer_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['owa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_inner_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['iwa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_mask_nyquist = (dh_outer_nyquist - dh_inner_nyquist).astype('bool')

dh_size = len(np.where(luvoir.dh_mask != 0)[0])
dh_size_nyquist = len(np.where(dh_mask_nyquist != 0)[0])
dh_index = np.where(luvoir.dh_mask != 0)[0]
dh_index_nyquist = np.where(dh_mask_nyquist != 0)[0]

In [ ]:
### Rebinning everything to the right sampling
#
E0_LOWFS = np.zeros([N_pup_z*N_pup_z,1,2])
E0_LOWFS[:,0,0] = Efield_ref_LOWFS.real
E0_LOWFS[:,0,1] = Efield_ref_LOWFS.imag
E0_OBWFS = np.zeros([N_pup_z*N_pup_z,1,2])
E0_OBWFS[:,0,0] = Efield_ref_OBWFS.real
E0_OBWFS[:,0,1] = Efield_ref_OBWFS.imag
E0_coron = np.zeros([nimg*nimg,1,2])
E0_coron[:,0,0] = Efield_ref.real
E0_coron[:,0,1] = Efield_ref.imag
E0_coron_nyquist = np.zeros([n_nyquist*n_nyquist,1,2])
tmp0 = hcipy.interpolation.make_linear_interpolator_separated(Efield_ref, grid=grid_test)
Efield_ref_nyquist = (luvoir.sampling/nyquist_sampling)**2*tmp0(grid_det_subsample)
E0_coron_nyquist[:,0,0] = Efield_ref_nyquist.real
E0_coron_nyquist[:,0,1] = Efield_ref_nyquist.imag
E0_coron_DH = np.zeros([dh_size,1,2])
E0_coron_DH[:,0,0] = Efield_ref.real[dh_index]
E0_coron_DH[:,0,1] = Efield_ref.imag[dh_index]
E0_coron_DH_nyquist = np.zeros([dh_size_nyquist,1,2])
E0_coron_DH_nyquist[:,0,0] = Efield_ref_nyquist.real[dh_index_nyquist]
E0_coron_DH_nyquist[:,0,1] = Efield_ref_nyquist.real[dh_index_nyquist]

In [ ]:
G_coron_LO_nyquist = np.zeros([n_nyquist*n_nyquist,2,n_LO-1])
for pp in range(1, n_LO):
    tmp0 = G_LO_real[pp] + 1j*G_LO_imag[pp]
    tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
    tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
    G_coron_LO_nyquist[:,0,pp-1] = tmp2.real - Efield_ref_nyquist.real
    G_coron_LO_nyquist[:,1,pp-1] = tmp2.real - Efield_ref_nyquist.imag

In [ ]:
G_coron_MID_nyquist= np.zeros([n_nyquist*n_nyquist,2,n_MID])
for pp in range(0, n_MID):
    tmp0 = G_MID_real[pp] + 1j*G_MID_imag[pp]
    tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
    tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
    G_coron_MID_nyquist[:,0,pp] = tmp2.real - Efield_ref_nyquist.real
    G_coron_MID_nyquist[:,1,pp] = tmp2.real - Efield_ref_nyquist.imag
G_coron_HI_nyquist= np.zeros([n_nyquist*n_nyquist,2,n_HI])
for pp in range(0, n_HI):
    tmp0 = G_HI_real[pp] + 1j*G_HI_imag[pp]
    tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
    tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
    G_coron_HI_nyquist[:,0,pp] = tmp2.real - Efield_ref_nyquist.real
    G_coron_HI_nyquist[:,1,pp] = tmp2.real - Efield_ref_nyquist.imag

G_coron_LO_DH = np.zeros([dh_size,2,n_LO-1])
for pp in range(1, n_LO):
    G_coron_LO_DH[:,0,pp-1] = G_LO_real[pp,dh_index] - Efield_ref.real[dh_index]
    G_coron_LO_DH[:,1,pp-1] = G_LO_imag[pp,dh_index] - Efield_ref.imag[dh_index]
G_coron_MID_DH= np.zeros([dh_size,2,n_MID])
for pp in range(0, n_MID):
    G_coron_MID_DH[:,0,pp] = G_MID_real[pp,dh_index] - Efield_ref.real[dh_index]
    G_coron_MID_DH[:,1,pp] = G_MID_imag[pp,dh_index] - Efield_ref.imag[dh_index]
G_coron_HI_DH= np.zeros([dh_size,2,n_HI])
for pp in range(0, n_HI):
    G_coron_HI_DH[:,0,pp] = G_HI_real[pp,dh_index] - Efield_ref.real[dh_index]
    G_coron_HI_DH[:,1,pp] = G_HI_imag[pp,dh_index] - Efield_ref.imag[dh_index]

In [ ]:
G_coron_LO_DH_nyquist = np.zeros([dh_size_nyquist,2,n_LO-1])
for pp in range(1, n_LO):
    tmp0 = G_LO_real[pp] + 1j*G_LO_imag[pp]
    tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
    tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
    G_coron_LO_DH_nyquist[:,0,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.real[dh_index_nyquist]
    G_coron_LO_DH_nyquist[:,1,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.imag[dh_index_nyquist]
G_coron_MID_DH_nyquist= np.zeros([dh_size_nyquist,2,n_MID])
for pp in range(0, n_MID):
    tmp0 = G_MID_real[pp] + 1j*G_MID_imag[pp]
    tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
    tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
    G_coron_MID_DH_nyquist[:,0,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.real[dh_index_nyquist]
    G_coron_MID_DH_nyquist[:,1,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.imag[dh_index_nyquist]
G_coron_HI_DH_nyquist= np.zeros([dh_size_nyquist,2,n_HI])
for pp in range(0, n_HI):
    tmp0 = G_HI_real[pp] + 1j*G_HI_imag[pp]
    tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
    tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
    G_coron_HI_DH_nyquist[:,0,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.real[dh_index_nyquist]
    G_coron_HI_DH_nyquist[:,1,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.imag[dh_index_nyquist]

In [ ]:
G_coron_LO = np.zeros([nimg*nimg,2,n_LO-1])
for pp in range(1, n_LO):
    G_coron_LO[:,0,pp-1] = G_LO_real[pp] - Efield_ref.real
    G_coron_LO[:,1,pp-1] = G_LO_imag[pp] - Efield_ref.imag
G_coron_MID= np.zeros([nimg*nimg,2,n_MID])
for pp in range(0, n_MID):
    G_coron_MID[:,0,pp] = G_MID_real[pp] - Efield_ref.real
    G_coron_MID[:,1,pp] = G_MID_imag[pp] - Efield_ref.imag
G_coron_HI= np.zeros([nimg*nimg,2,n_HI])
for pp in range(0, n_HI):
    G_coron_HI[:,0,pp] = G_HI_real[pp] - Efield_ref.real
    G_coron_HI[:,1,pp] = G_HI_imag[pp] - Efield_ref.imag

In [ ]:
G_LOWFS = np.zeros([N_pup_z*N_pup_z,2,n_LO-1])
for pp in range(1, n_LO):
    G_LOWFS[:,0,pp-1] = G_LOWFS_real[pp]*z_pup_downsample - Efield_ref_LOWFS.real
    G_LOWFS[:,1,pp-1] = G_LOWFS_imag[pp]*z_pup_downsample - Efield_ref_LOWFS.imag
G_OBWFS= np.zeros([N_pup_z*N_pup_z,2,n_MID])
for pp in range(0, n_MID):
    G_OBWFS[:,0,pp] = G_OBWFS_real[pp]*z_pup_downsample - Efield_ref_OBWFS.real
    G_OBWFS[:,1,pp] = G_OBWFS_imag[pp]*z_pup_downsample - Efield_ref_OBWFS.imag

In [ ]:
def req_closedloop_calc_recursive(Gcoro, Gsensor, E0coro, E0sensor, Dcoro, Dsensor, t_exp, flux, Q, Niter, dh_mask,
                                  norm):
    P = np.zeros(Q.shape)  # WFE modes covariance estimate
    r = Gsensor.shape[2]
    N = Gsensor.shape[0]
    N_img = Gcoro.shape[0]
    c = 1
    # Iterations of ALGORITHM 1
    contrast_hist = np.zeros(Niter)
    intensity_WFS_hist = np.zeros(Niter)
    cal_I_hist = np.zeros(Niter)
    eps_hist = np.zeros([Niter, r])
    averaged_hist = np.zeros(Niter)
    contrasts = []
    for pp in range(Niter):
        eps = np.random.multivariate_normal(np.zeros(r), P + Q * t_exp).reshape((1, 1, r))  # random modes
        G_eps = np.sum(Gsensor * eps, axis=2).reshape((N, 1, 2 * c)) + E0sensor  # electric field
        G_eps_squared = np.sum(G_eps * G_eps, axis=2, keepdims=True)
        G_eps_G = np.matmul(G_eps, Gsensor)
        G_eps_G_scaled = G_eps_G / np.sqrt(G_eps_squared + Dsensor / flux / t_exp)  # trick to save RAM
        cal_I = 4 * flux * t_exp * np.einsum("ijk,ijl->kl", G_eps_G_scaled, G_eps_G_scaled)  # information matrix
        P = np.linalg.inv(np.linalg.inv(P + Q * t_exp / 2) + cal_I)
        #         P = np.linalg.inv(cal_I)

        # Coronagraph
        G_eps_coron = np.sum(Gcoro * eps, axis=2).reshape((N_img, 1, 2 * c)) + E0coro
        G_eps_coron_squared = np.sum(G_eps_coron * G_eps_coron, axis=2, keepdims=True)
        intensity = G_eps_coron_squared * flux * t_exp + Dcoro

        # Wavefront sensor
        intensity_WFS = G_eps_squared * flux * t_exp + Dsensor

        # Archive
        test_DH0 = intensity[:, 0, 0] * luvoir.dh_mask
        test_DH = np.mean(test_DH0[np.where(test_DH0 != 0)])
        contrasts.append(test_DH / flux / t_exp / norm)
        intensity_WFS_hist[pp] = np.sum(intensity_WFS) / flux
        cal_I_hist[pp] = np.mean(cal_I) / flux
        eps_hist[pp] = eps
        averaged_hist[pp] = np.mean(contrasts)
        #         print("est. contrast", np.mean(contrasts))

        outputs = {'intensity_WFS_hist': intensity_WFS_hist,
                   'cal_I_hist': cal_I_hist,
                   'eps_hist': eps_hist,
                   'averaged_hist': averaged_hist,
                   'contrasts': contrasts}
    return outputs

In [ ]:
def req_closedloop_calc_batch(Gcoro, Gsensor, E0coro, E0sensor, Dcoro, Dsensor, t_exp, flux, Q, Niter, dh_mask, norm):
    P = np.zeros(Q.shape)  # WFE modes covariance estimate
    r = Gsensor.shape[2]
    N = Gsensor.shape[0]
    N_img = Gcoro.shape[0]
    c = 1
    # Iterations of ALGORITHM 1
    contrast_hist = np.zeros(Niter)
    intensity_WFS_hist = np.zeros(Niter)
    cal_I_hist = np.zeros(Niter)
    eps_hist = np.zeros([Niter, r])
    averaged_hist = np.zeros(Niter)
    contrasts = []
    for pp in range(Niter):
        eps = np.random.multivariate_normal(np.zeros(r), P + Q * t_exp).reshape((1, 1, r))  # random modes
        G_eps = np.sum(Gsensor * eps, axis=2).reshape((N, 1, 2 * c)) + E0sensor  # electric field
        G_eps_squared = np.sum(G_eps * G_eps, axis=2, keepdims=True)
        G_eps_G = np.matmul(G_eps, Gsensor)
        G_eps_G_scaled = G_eps_G / np.sqrt(G_eps_squared + Dsensor / flux / t_exp)  # trick to save RAM
        cal_I = 4 * flux * t_exp * np.einsum("ijk,ijl->kl", G_eps_G_scaled, G_eps_G_scaled)  # information matrix
        #         P = np.linalg.inv(np.linalg.inv(P+Q*t_exp/2) + cal_I)
        P = np.linalg.pinv(cal_I)

        # Coronagraph
        G_eps_coron = np.sum(Gcoro * eps, axis=2).reshape((N_img, 1, 2 * c)) + E0coro
        G_eps_coron_squared = np.sum(G_eps_coron * G_eps_coron, axis=2, keepdims=True)
        intensity = G_eps_coron_squared * flux * t_exp + Dcoro

        # Wavefront sensor
        intensity_WFS = G_eps_squared * flux * t_exp + Dsensor

        # Archive
        test_DH0 = intensity[:, 0, 0] * luvoir.dh_mask
        test_DH = np.mean(test_DH0[np.where(test_DH0 != 0)])
        contrasts.append(test_DH / flux / t_exp / norm)
        intensity_WFS_hist[pp] = np.sum(intensity_WFS) / flux
        cal_I_hist[pp] = np.mean(cal_I) / flux
        eps_hist[pp] = eps
        averaged_hist[pp] = np.mean(contrasts)
    #         print("est. contrast", np.mean(contrasts))
    #         print("est. contrast", np.mean(contrasts))

    outputs = {'intensity_WFS_hist': intensity_WFS_hist,
               'cal_I_hist': cal_I_hist,
               'eps_hist': eps_hist,
               'averaged_hist': averaged_hist,
               'contrasts': contrasts}

    return outputs

In [ ]:
flux = Nph
QLO = np.diag(np.asarray(mu_mapLO**2))
QMID = np.diag(np.asarray(mu_mapMID**2))
QHI = np.diag(np.asarray(mu_mapHI**2))

# Running a bunch of tests for time series

Ntimes = 20
TimeMinus = -2
TimePlus = 3.5
Nwavescale = 8
WaveScaleMinus = -2
WaveScalePlus = 1
Nflux = 3
fluxPlus = 10
fluxMinus = 0

timeVec = np.logspace(TimeMinus,TimePlus,Ntimes)
WaveVec = np.logspace(WaveScaleMinus,WaveScalePlus,Nwavescale)
fluxVec = np.linspace(fluxMinus,fluxPlus,Nflux)
wavescaleVec = np.logspace(WaveScaleMinus,WaveScalePlus,Nwavescale)

In [ ]:
niter = 2

print('LO modes with batch LOWFS and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus, fluxPlus, Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_batch(G_coron_LO, G_LOWFS, E0_coron, E0_LOWFS, dark_current+CIC/tscale,
                                             dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QLO,
                                             niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

In [ ]:
res_line = np.reshape(res, [Ntimes*Nwavescale*Nflux])
text_files_name = overall_dir + '/LO_LOWFS_Batch_dark_' + np.str(dark_current) + '_CIC_' + np.str(CIC) + '.csv'
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)


print('LO modes with recursive LOWFS and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus, fluxPlus, Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_recursive(G_coron_LO, G_LOWFS, E0_coron, E0_LOWFS, dark_current+CIC/tscale,
                                                 dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QLO,
                                                 niter, luvoir.dh_mask,norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'LO_LOWFS_Recursive_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)


print('LO modes with batch Nyquist DH and noise')

In [ ]:
timer1 = time.time()


res = np.zeros([Ntimes,Nwavescale,Nflux,1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus,fluxPlus,Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_batch(G_coron_LO, G_coron_LO_DH_nyquist, E0_coron, E0_coron_DH_nyquist,
                                             dark_current+CIC/tscale, dark_current+CIC/tscale, tscale,
                                             flux*Starfactor, wavescale**2*QLO, niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'LO_DH_Batch_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)

print('LO modes with recursive Nyquist DH and noise')

timer1 = time.time()

In [ ]:
res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus,fluxPlus,Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_recursive(G_coron_LO, G_coron_LO_DH_nyquist, E0_coron, E0_coron_DH_nyquist,
                                                 dark_current+CIC/tscale, dark_current+CIC/tscale, tscale,
                                                 flux*Starfactor, wavescale**2*QLO, niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

#  takes a long time to run

In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'LO_DH_Recursive_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)

print('MID modes with batch OBWFS and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus, fluxPlus, Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_batch(G_coron_MID, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
                                             dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QMID,
                                             niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'MID_OBWFS_Batch_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)


print('MID modes with recursive OBWFS and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus,fluxPlus,Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_recursive(G_coron_MID, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
                                                 dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QMID,
                                                 niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'MID_OBWFS_Recursive_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)


print('MID modes with batch Nyquist DH and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus, fluxPlus, Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_batch(G_coron_MID, G_coron_MID_DH_nyquist, E0_coron, E0_coron_DH_nyquist, dark_current+CIC/tscale, dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QMID, niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1


In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'MID_DH_Batch_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)

print('MID modes with recursive Nyquist DH and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus, fluxPlus, Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_recursive(G_coron_MID, G_coron_MID_DH_nyquist, E0_coron, E0_coron_DH_nyquist, dark_current+CIC/tscale, dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QMID, niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1


In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'MID_DH_Recursive_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)

print('HI modes with batch Nyquist DH and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus, fluxPlus, Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_batch(G_coron_HI, G_coron_HI_DH_nyquist, E0_coron, E0_coron_DH_nyquist, dark_current+CIC/tscale, dark_current+CIC/tscale, tscale, flux*Starfactor, wavescale**2*QHI, niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1


In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'HI_DH_Batch_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)

print('HI modes with recursive Nyquist DH and noise')

timer1 = time.time()


res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
pp = 0
for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
    qq = 0
    print(tscale)
    for wavescale in np.logspace(WaveScaleMinus, WaveScalePlus, Nwavescale):
        rr = 0
        for StarMag in np.linspace(fluxMinus,fluxPlus,Nflux):
            Starfactor = 10**(-StarMag/2.5)
            tmp0 = req_closedloop_calc_recursive(G_coron_HI, G_coron_HI_DH_nyquist, E0_coron, E0_coron_DH_nyquist,
                                                 dark_current+CIC/tscale, dark_current+CIC/tscale, tscale,
                                                 flux*Starfactor, wavescale**2*QHI, niter, luvoir.dh_mask, norm)
            tmp1 = tmp0['averaged_hist']
            n_tmp1 = len(tmp1)
            res[pp,qq,rr] =  np.mean(tmp1[np.int(n_tmp1/2):n_tmp1]) - contrast_floor
            rr = rr + 1
        qq = qq + 1
    pp = pp + 1

In [ ]:
res_line = np.reshape(res,[Ntimes*Nwavescale*Nflux])
text_files_name = os.path.join(overall_dir, f'HI_DH_Recursive_dark_{dark_current}_CIC_{CIC}.csv')
np.savetxt(text_files_name, res_line, delimiter=",")

timer2 = time.time()
print(timer2 - timer1)